In [1]:
import django 
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from sch.models import *
from sch.actions import *

In [3]:
slot = Slot.objects.get(slug__contains='2022-11-05-RS')
def resolveTurnaroundWithinDay (turnaroundSlot):
        emp = turnaroundSlot.employee
        if emp.evening_pref:
            wd = turnaroundSlot.workday
            sfta = turnaroundSlot.shift
        elif emp.evening_pref == False:
            wd = turnaroundSlot.workday.prevWD()
            sfta = Slot.objects.get(workday=wd,employee=emp)
        
        sfts = emp.shifts_available.all()
        print('shifts_available',sfts)
        noIncompatibles = []
        for sft in sfts:
            if not Slot.objects.incompatible_slots(workday=wd,shift=sft).filter(employee=emp).exists():
                noIncompatibles += [sft]
                print(sft, noIncompatibles)
        List = []
        for sft in noIncompatibles:
            checking = Slot.objects.filter(workday=wd,shift=sft)
            print(checking)
            if checking.exists():
                if sfta in checking.first().employee.shifts_available.all():
                    if not Slot.objects.incompatible_slots(workday=wd,shift=sfta).filter(employee=checking.first().employee).exists():
                        List += [sfta]  
        print(List)  
        if len(List) == 1:
            return List[0]  
        return List[random.randint(0,len(List)-1)]

In [13]:
from django.db.models import IntegerField 

In [31]:
emp = Employee.objects.get(name='Josh')
print("Avg Agg Shift Pref Score:",
      emp.agg_avg_shift_pref_score)
print("WeeklyHours, week 43:",
      emp.weekly_hours(2022,43), 'hrs')
print("WeeklyHours Percent:",
      emp.weekly_hours_perc(2022,43) * 100, "%")

Avg Agg Shift Pref Score: -0.2222222222222222
WeeklyHours, week 43: 30.0 hrs
WeeklyHours Percent: 120.0 %


In [32]:
emp.templated_days

<QuerySet [<ShiftTemplate: SatA:MI Template>, <ShiftTemplate: SunB:MI Template>, <ShiftTemplate: SatB:MI Template>, <ShiftTemplate: SunB:MI Template>]>

In [8]:
wds = Workday.objects.filter(date__year=2022,iweek=3).first()
wds.who_can_fill(Shift.objects.get(name='MI'))


<EmployeeManager [<Employee: Josh>, <Employee: Sabrina>, <Employee: Brianna>, <Employee: Esperanza>, <Employee: Elisia>, <Employee: Jaden>, <Employee: Trisha>, <Employee: Tiffany>, <Employee: David>, <Employee: Nicki>, <Employee: Leslie>]>

In [40]:
for sft in Shift.objects.filter(start__hour__gte=20):
    sft.group = 'XT'
    sft.save()

In [48]:
slts = Slot.objects.filter(shift__name='S',workday__date__year=2022,workday__iweek=43)

print(slts)

<SlotManager []>
